# Experiment Introduction

Background, 
In order to do ablation experiment, we have diffculty like
1. most of them are same, we have to repeat many times.
2. too many model, config, code hard to manage and save. too messy


For all kind of SSL training workflow, we have to define the hyperparameters includes 4 aspect,
Dataset
Model
Training
Saving COnfig



How to Use this Experiment?



In [ ]:
from lightly.utils.benchmarking import BenchmarkModule
from lightly.models import ResNetGenerator, modules, utils
import copy
import os
import time

import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision
from lightly.models import ResNetGenerator, modules, utils
from lightly.models.modules import heads
from lightly.loss import (
    BarlowTwinsLoss,
    DCLLoss,
    DCLWLoss,
    DINOLoss,
    NegativeCosineSimilarity,
    NTXentLoss,
    SwaVLoss,
    memory_bank,
)
class BarlowTwinsModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )
        # use a 2-layer projection head for cifar10 as described in the paper
        self.projection_head = heads.ProjectionHead(
            [
                (512, 2048, nn.BatchNorm1d(2048), nn.ReLU(inplace=True)),
                (2048, 2048, None, None),
            ]
        )

        self.criterion = BarlowTwinsLoss(gather_distributed=gather_distributed)

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2 * lr_factor, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]
# Set to True to enable Distributed Data Parallel training.
distributed = False
# gpus, otherwise only features from the same gpu are used.
gather_distributed = False
# benchmark
n_runs = 1  # optional, increase to create multiple runs and report mean + std
batch_size = 512
lr_factor = batch_size / 128  # scales the learning rate linearly with batch size
max_epochs = 50



# Library

In [1]:
# Import config
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoSSL")
import yaml
from torchvision.transforms import RandomRotation,GaussianBlur,ColorJitter
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg ,Toymodel, pipe_model 
from autoSSL.utils import embedding_feature,ck_callback,dict2transformer,trans2multi,join_dir,ContinuousCSVLogger  
from autoSSL.data import PipeDataset
from autoSSL.train import Trainer

from lightly.transforms import SimCLRTransform
from lightly.transforms.rotation import random_rotation_transform
import torchvision.transforms as T
import pytorch_lightning as pl
pl.seed_everything(0)

C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


0

# Import Global Baseline Config

In [2]:
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)

# Write your experiment notebook name here
global_config["experiment"]="lightly_baseline"   
global_config["dataset_dir"]="../Datasets/cifar10/train/"
global_config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,None,None]]    

global_config["loss_func"]="BarlowTwinsLoss"     
global_config["view_model"]="None"     
global_config["view"]=2 
global_config["stop_gradient"]=False   
global_config["optimizer"]="SGD"      
global_config["schedule"]="cos"   
global_config["model"]="Toymodel"
global_config["batch_size"]=128
global_config["input_size"]=32
#global_config["backbone"]="resnet18_pretrained"

# Define global Training Augmentation
global_SSL_augmentation=T.Compose([
    T.RandomResizedCrop(size=global_config["input_size"], scale=(0.08, 1.0)),
    T.RandomApply([T.RandomRotation(degrees=90)], p=0),
    T.RandomHorizontalFlip(p= 0.5),
    T.RandomVerticalFlip(p=0),
    T.RandomApply([T.ColorJitter(
            brightness=0.4,contrast=0.4,saturation=0.4, hue=0.1)], p=0.8),
    T.RandomGrayscale(p=0.2),
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Define the Testing Augmentation
test_SSL_augmentation=T.Compose([
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

p_knndata= PipeDataset(input_dir=global_config["path_to_test_cifar10"], 
        augmentation=trans2multi(test_SSL_augmentation,view=1), 
        batch_size=global_config["batch_size"],num_workers=global_config["num_workers"]).dataloader
p_knndata=[p_knndata,10]  # The second number is the classes number of this datasets


# Grid Run Config

## Config2: Use SGD + Cos

In [ ]:
for baseline in [1]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["name"]=f"BT_PIPE_SGD+COS4"
    config["batch_size"]=512
    config["max_epochs"]=50
    config["optimizer"]="SGD"      
    config["schedule"]="cos"   
    config["backbone"]="resnet18"   
    config["dataset_dir"]="../Datasets/cifar10/train/"
    
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    
    del pdata
    del pmodel
    del trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\lightly_baseline\BT_PIPE_SGD+COS4 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type            | Params
--------------------------------------------

Epoch 2:   0%|                                                                         | 0/97 [00:00<?, ?it/s, v_num=2]

## Check Perfect

In [4]:

for baseline in [1]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation 
    config["name"]=f"BT_Baseline_SGD+COS+res_18_cifar3"
    config["batch_size"]=512
    config["max_epochs"]=50
    config["optimizer"]="SGD"      
    config["schedule"]="cos"   
    config["backbone"]="resnet18"   
    config["dataset_dir"]="../Datasets/cifar10/train/"
    # THIS IS THE CODE TO LOAD DATASET
    
    pdata= PipeDataset(config=config,augmentation=trans2multi(global_SSL_augmentation,view=2))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    #pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    pmodel=BarlowTwinsModel(p_knndata[0],10)
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    
    del pdata
    del pmodel
    del trainer
    
    

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\lightly_baseline\BT_Baseline_SGD+COS+res_18_cifar3 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type            | Params
---------------------------

Epoch 4: 100%|████████████████████████████████████████████████████████████████| 97/97 [00:27<00:00,  3.56it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|████████████████████████████████████████████| 97/97 [00:28<00:00,  3.42it/s, v_num=0, kNN_accuracy=27.60]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|███████████████████████████████████████████| 97/97 [00:30<00:00,  3.21it/s, v_num=0, kNN_accuracy=32.00]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|███████████████████████████████████████████| 97/97 [00:29<00:00,  3.32it/s, v_num=0, kNN_accuracy=38.00]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|███████████████████████████████████████████| 97/97 [00:29<00:00,  3.30it/s, v_num=0, kNN_accuracy=44.60]
Validation: 0it [00:00, ?it/s]
Epoch 29: 100%|███████████████████████████████████████████| 97/97 [00:28<00:00,  3.45it/s, v_num=0, kNN_accuracy=49.10]
Validation: 0it [00:00, ?it/s]
Epoch 34: 100%|███████████████████████████████████████████| 97/97 [00:27<00:00,  3.52it/s, v_n

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|███████████████████████████████████████████| 97/97 [00:45<00:00,  2.14it/s, v_num=0, kNN_accuracy=60.30]


## Improved Trainer

In [7]:
 # Use SimCLR augmentations
from lightly.transforms import SimCLRTransform
simclr_transform = SimCLRTransform(
    input_size=32,
    cj_strength=0.5,
    gaussian_blur=0.0,
)


# THIS IS THE CODE TO Monitor the KNN accuracy for each epoch
p_knndata= PipeDataset(input_dir=global_config["path_to_test_cifar10"], 
        augmentation=dict2transformer(test_SSL_augmentation,view=1), 
        batch_size=512,num_workers=global_config["num_workers"]).dataloader
p_knndata=[p_knndata,10]  # The second number is the classes number of this datasets

In [8]:
from autoSSL.utils import ck_callback, join_dir,ContinuousCSVLogger
pl.seed_everything(0)
for baseline in [1]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"BT_Baseline_trainer_SGD+COS+res_18_cifar"
    config["batch_size"]=512
    config["max_epochs"]=50
    config["optimizer"]="SGD"      
    config["schedule"]="cos"   
    config["backbone"]="resnet18"   
    config["dataset_dir"]="../Datasets/cifar10/train/"
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=simclr_transform)
    config["log_dir"] = join_dir(config["checkpoint_dir"], config["experiment"], config["name"])
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    #pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    pmodel=BarlowTwinsModel(p_knndata[0],10)
 
   
    trainer = pl.Trainer(
                max_epochs=config["max_epochs"],
                devices=1,
                accelerator="gpu",
                #default_root_dir=logs_root_dir,
                strategy="auto",
                sync_batchnorm=False,
                logger=ContinuousCSVLogger(save_dir=config["log_dir"]),
                callbacks=[ck_callback(config["log_dir"])],
                check_val_every_n_epoch =5,
            )

    # Use this if you want to START a train
    # trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    #trainer.fit(pmodel, pdata.dataloader,)  
    trainer.fit(pmodel, pdata.dataloader,val_dataloaders=pmodel.dataloader_kNN)  
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    
    del pdata
    del pmodel
    del trainer
    
    

Global seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type            | Params
----------------------------------------------------
0 | backbone        | Sequential      | 11.2 M
1 | projection_head | ProjectionHead  | 5.2 M 
2 | criterion       | BarlowTwinsLoss | 0     
----------------------------------------------------
16.4 M    Trainable params
0         Non-trainable params
16.4 M    Total params
65.671    Total estimated model params size (

Epoch 4: 100%|████████████████████████████████████████████████████████████████| 97/97 [00:27<00:00,  3.52it/s, v_num=3]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|████████████████████████████████████████████| 97/97 [00:26<00:00,  3.61it/s, v_num=3, kNN_accuracy=27.20]
Validation: 0it [00:00, ?it/s]
Epoch 11:   0%|                                                    | 0/97 [00:00<?, ?it/s, v_num=3, kNN_accuracy=32.60]

C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\trainer\call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000025A62F8E9D0>
Traceback (most recent call last):
  File "C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\torch\utils\data\dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\torch\utils\data\dataloader.py", line 1436, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


# Evaluation

In [8]:

test_augmentation = {
    'RandomResizedCrop': {'size': (global_config["input_size"], global_config["input_size"])},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}
collate =pipe_collate(address="experiment_checkpoints/paper_barlowtwins/", reg="BT_Baseline_512_[0-9]+")

pdata = PipeDataset(input_dir=global_config["path_to_test_cifar10"],
    augmentation=dict2transformer(test_augmentation,view=1))


Collating the models' (evaluating) information to experiment_checkpoints/paper_barlowtwins/BT_Baseline_512_[0-9]+.csv


In [9]:
aaa=eval_linear(pdata, models=collate, device=global_config["device"])


Load the training and testing dataset


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.38s/it]


In [ ]:
aaa